In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import trl
trl.trainer.ConstantLengthDataset
from trl.extras.dataset_formatting import conversations_formatting_function

from datasets import load_dataset, load_from_disk, DatasetDict
from transformers.pipelines.pt_utils import KeyDataset

from transformers import AutoTokenizer, AutoModelForCausalLM
import yaml
from pathlib import Path
import os

/home/mmordig/reinforcement/alphageometry/LLM_finetuner/verbalization_venv3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


/home/mmordig/reinforcement/alphageometry/LLM_finetuner/verbalization_venv3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:

dataset_dir = "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/datasets/alpha_geo_small_arrow"
dataset = load_dataset(dataset_dir)

In [5]:
dataset
dataset["train"][0]
dataset["train"][:2]
dataset


DatasetDict({
    train: Dataset({
        features: ['nl_statement', 'fl_statement'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['nl_statement', 'fl_statement'],
        num_rows: 20
    })
    test: Dataset({
        features: ['nl_statement', 'fl_statement'],
        num_rows: 20
    })
})

In [6]:
from LLM_finetuner.dataset import replace_geometry_words_with_tokens, format_question_answer_single


extra_tokens_file = Path(os.path.expanduser("~/reinforcement/alphageometry/assets/def-patterns-desc.yml"))
def_to_desc = yaml.safe_load(extra_tokens_file.read_text())
# eq_triangle, ieq_triangle, so replace by longest first
geom_tokens = sorted(list(def_to_desc.keys()), key=lambda x: -len(x))
print(geom_tokens)

dataset = dataset.map(lambda x: {"fl_statement": replace_geometry_words_with_tokens(x["fl_statement"], geom_tokens)}, num_proc=8)
dataset["train"][:2]

['eqdia_quadrangle', 'intersection_cc', 'intersection_lc', 'intersection_ll', 'intersection_lp', 'intersection_lt', 'intersection_pp', 'intersection_tt', 'angle_bisector', 'eq_quadrangle', 'parallelogram', 'angle_mirror', 'circumcenter', 'eq_trapezoid', 'iso_triangle', 'ieq_triangle', 'eq_triangle', 'orthocenter', 'r_trapezoid', 'cc_tangent0', 'eqdistance', 'ninepoints', 'lc_tangent', 'quadrangle', 'r_triangle', 'triangle12', 'trisegment', 'cc_tangent', 'incenter2', 'excenter2', 'on_aline2', 'on_circle', 'rectangle', 'trapezoid', 'on_opline', 'on_circum', 'eqangle2', 'incenter', 'excenter', 'centroid', 'midpoint', 'on_aline', 'on_bline', 'on_pline', 'on_tline', 'pentagon', 'triangle', 'eqangle3', 'nsquare', 'on_line', 'psquare', 'reflect', 's_angle', 'segment', 'isquare', 'trisect', 'tangent', 'circle', 'mirror', 'square', 'on_dia', 'risos', 'shift', 'e5128', 'free', 'foot', '2l1c', '3peq']


{'nl_statement': ['Points B, A, C are defined such that triangle ABC is an equilateral triangle. Define points D, F, & E such that E, D, and F is a right angle isosceles triangle with the right angle at D. Point G is defined such that G, A, E are three consecutive vertices of a square. Circle centered at C with radius CE intersects circle centered at G with radius GE at H and E. Points I and J are defined such that line IC and line JC are the two tangents to circle centered at B with radius BF at point I and J respectively.. Define point K such that line DI and line AG are parallel. line JK perpendicular to line DI. line EK perpendicular to line AG. line JK meets line EK at the point K.',
  'Points A, D, B, C are defined such that A, B, C, and D is a trapezoid where line AD is equal to line BC. E is a point. F is defined such that line FC is touching to circle centered at A with radius AC at the point F. Define points G, J, I, & H such that J is located at the incenter of triangle ECB 

In [45]:
# from transformers import DataCollatorWithPadding
?tokenizer.pad

Signature:
tokenizer.pad(
    encoded_inputs: Union[transformers.tokenization_utils_base.BatchEncoding, List[transformers.tokenization_utils_base.BatchEncoding], Dict[str, List[int]], Dict[str, List[List[int]]], List[Dict[str, List[int]]]],
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = True,
    max_length: Optional[int] = None,
    pad_to_multiple_of: Optional[int] = None,
    return_attention_mask: Optional[bool] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    verbose: bool = True,
) -> transformers.tokenization_utils_base.BatchEncoding
Docstring:
Pad a single encoded input or a batch of encoded inputs up to predefined length or to the max sequence length
in the batch.

Padding side (left/right) padding token ids are defined at the tokenizer level (with `self.padding_side`,
`self.pad_token_id` and `self.pad_token_type_id`).

Please note that with a fast tokenizer, using the `__call__` method is faster t

In [ ]:
# see transformers.Trainer.evaluation_loop
# from transformers import Trainer, TrainingArguments, Seq2SeqTrainer

import sys
from LLM_finetuner.sft_finetuning import main as get_trainer
import LLM_finetuner
LLM_finetuner.sft_finetuning.TRL_USE_RICH = False

sys.argv = ["--overwrite_output_dir", "--use_peft", "--per_device_train_batch_size", "64", "--per_device_eval_batch_size", "64", "--model_name_or_path", "gpt2", "--eval_steps", "10", "--evaluation_strategy", "steps", "--max_eval_samples", "2", "--explicit_eos_str", "[END]", "--extra_tokens_file", "/home/mmordig/reinforcement/alphageometry/assets/def-patterns-desc.yml", "--output_dir", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/training/debug112/{model_name}_{max_train_samples}ex_peft{use_peft}", "--dataset_name", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/runs/verbalization/datasets/alpha_geo_processed", "--config", "/home/mmordig/reinforcement/alphageometry/LLM_finetuner/trl_sft_config.yml", "--max_train_samples", "10", "--num_train_epochs", "100000",]
sys.argv += ["--include_inputs_for_metrics"]
# sys.argv += ["--dataloader_num_workers: 0"]
trainer, resume_checkpoint = get_trainer()


In [ ]:
import wandb
def compute_metrics(eval_results):
    breakpoint()
    predictions = eval_results.predictions
    inputs = eval_results.inputs
    labels = eval_results.label_ids
    # print(predictions)
    print(inputs[0])
    print(labels[0])
    print(predictions[0])
    predictions = trainer.tokenizer.batch_decode(predictions)
    inputs = trainer.tokenizer.batch_decode(inputs)
    labels = trainer.tokenizer.batch_decode(labels)
    print(predictions)
    print(labels)
    # print(predictions.shape)
    # return {"generated_text": predictions}
    # return {"generated_text2": wandb.Table(columns=["text"], data=[[p] for p in predictions])}
    return {"predictions": wandb.Table(columns=["input", "predicted"], data=[[i, p] for (i, p) in zip(inputs, predictions)])}
    
trainer.compute_metrics = compute_metrics
trainer.evaluate()

In [ ]:
# class TrainerWithGeneration(Trainer):
#     def generate_with_model(self):
#         model = self._wrap_model(self.model, training=False, dataloader=dataloader)
#         model.eval()
        

In [ ]:
import torch
from transformers import TrainerCallback

class MakePredictionsCallback(TrainerCallback):
    def __init__(self, max_generation_length=1024):
        # self.tokenizer = tokenizer
        self.max_generation_length = max_generation_length

    def on_epoch_end(self, args, state, control, model, eval_dataloader, **kwargs):
        model.eval()
        predictions = []
        for batch in eval_dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            with torch.no_grad():
                output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=self.max_generation_length)
            predictions.extend(output)
        # model.train()
        
        control.on_lo
        return control.

In [ ]:
import torch
import torch.nn as nn
from contextlib import nullcontext

# from torchdistx.fake import fake_mode
# with fake_mode():
with nullcontext():
    model = nn.Sequential(
        nn.Conv2d(1, 20, 5),
        nn.ReLU(),
        nn.Conv2d(20, 64, 5),
        nn.ReLU()
    )

    total_activations = 0

    def count_activations_hook(module, input, output):
        global total_activations
        activations = output.numel()
        total_activations += activations
        print(f"{module.__class__.__name__} produced {activations} activations")

    for layer in model.children():
        layer.register_forward_hook(count_activations_hook)

    input_tensor = torch.randn(1, 1, 28, 28)
    model(input_tensor)
    print(f"Total activations: {total_activations}")


In [ ]:
import torch
# model_name = "meta-llama/Llama-2-7b-chat-hf"
from accelerate import init_empty_weights
model_name = "gpt2"
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model(input_ids=torch.LongTensor([1, 2, 3])[None, :])

In [ ]:
dir(model)
# model.modules[0]
next(model.children())

In [ ]:
total_activations = 0

def count_activations_hook(module, input, output):
    global total_activations
    # output.shape[0] is the batch size
    # output.numel() gives the total number of elements in the output, for all batches
    # To get activations per single input, you would divide by output.shape[0]
    activations = output.numel()
    total_activations += activations
    print(f"{module.__class__.__name__} produced {activations} activations")

# Register hook for each layer
for layer in model.children():
    layer.register_forward_hook(count_activations_hook)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
print(accelerator.state)
print(str(accelerator))
print(f"{accelerator=}")

In [ ]:
from typing import List


    
# r'''one \"two\" 'three''''
# parse_htcondor_arguments('''"3 simple arguments"''')

from accelerate import init_empty_weights


# format_htcondor_arguments



In [ ]:
import accelerate
accelerate.wait_for_everyone()

In [ ]:
tuple("$(Cluster) $(Process) $(NumRanks)".split(" "))

In [ ]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
from LLM_finetuner.utils import add_new_tokens_with_average_init


extra_tokens_file = Path(os.path.expanduser("~/reinforcement/alphageometry/assets/def-patterns-desc.yml"))
def_to_desc = yaml.safe_load(extra_tokens_file.read_text())

add_new_tokens_with_average_init(model, tokenizer, def_to_desc)


In [ ]:
# def_to_desc
tokens = list(def_to_desc.keys())
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"The following tokens are already known, ignoring them: {[token for (token, id) in zip(tokens, token_ids) if id != tokenizer.unk_token_id]}")
def_to_desc = {token: def_to_desc[token] for (token, id) in zip(tokens, token_ids) if id == tokenizer.unk_token_id}
def_to_desc

In [ ]:
dataset_dir = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/runs/verbalization/datasets/alpha_geo_small_processed"
raw_datasets = load_from_disk(dataset_dir)
dataset = raw_datasets["test"]

In [ ]:
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr


model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
# model_name_or_path = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)#, add_eos_token=True)
# inputs = "Hello world"
inputs = [
    {"role": "system", "content": "System"},
    # must be ordered as user-assistant alternating sequence
    {"role": "user", "content": "Question1"},
    {"role": "assistant", "content": "Answer1"},
    {"role": "user", "content": "Question2"},
    {"role": "assistant", "content": "Answer2"},
]
print(tokenizer.apply_chat_template(inputs, tokenize=False, add_generation_prompt=True))
print("#"*80)

inputs = [
    {"role": "system", "content": "System"},
    # must be ordered as user-assistant alternating sequence
    {"role": "user", "content": "Question1"},
    {"role": "assistant", "content": "Answer1"},
    {"role": "user", "content": "Question2"},
    # {"role": "assistant", "content": "Answer2"},
]
print(tokenizer.apply_chat_template(inputs, tokenize=False, add_generation_prompt=True))

In [ ]:
query = "### Question: Points B, A, C are defined such that triangle ABC is an equilateral triangle. Define points D, F, & E such that E, D, and F is a right angle isosceles triangle with the right angle at D. Point G is defined such that G, A, E are three consecutive vertices of a square. Circle centered at C with radius CE intersects circle centered at G with radius GE at H and E. Points I and J are defined such that line IC and line JC are the two tangents to circle centered at B with radius BF at point I and J respectively.. Define point K such that line DI and line AG are parallel. line JK perpendicular to line DI. line EK perpendicular to line AG. line JK meets line EK at the point K. ### Answer: A B C = ieq_triangle A B C; D E F = risos D E F; G = psquare G A E; H = intersection_cc H C G E; I J = tangent I J C B F; K = intersection_tt K J D I E A G;"
tokenizer.decode(tokenizer(query)["input_ids"], skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [ ]:
tokenizer.tokenize(query)
" ".join(tokenizer.tokenize(query))

In [ ]:
import io


def print_to_str(x):
    temp_out = io.StringIO()
    print(x, file=temp_out)
    return temp_out.getvalue()
print(print_to_str("ss\na"))

In [ ]:
from LLM_finetuner.question_answer_utils import get_question_answer_to_chat_formatter

formatting_func = get_question_answer_to_chat_formatter(tokenizer, text_column="text")
print(f"Example format: {formatting_func(dataset[0])}")


In [ ]:
from geometry_translation.new.question_answer_utils import extract_question_answer



text_column = "text"
print(convert_question_answer_to_chat_format(dataset[0], text_column=text_column))

In [ ]:
encoded_inputs = tokenizer(, return_tensors="pt", add_special_tokens=True)
encoded_inputs

In [ ]:
tokenizer.decode(encoded_inputs["input_ids"][0])

In [ ]:
?tokenizer

In [ ]:
import textwrap
from datasets import load_dataset

def wrapping_print(x):
    print(textwrap.fill(x))
    


# dataset_dir = "/fast/mmordig/general_ai_rl/alphageom_project/alpha_geo_processed"
dataset_dir = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/geometry_translation/new/runs/verb_dataset"

raw_datasets = load_from_disk(dataset_dir)
dataset = raw_datasets["test"][:100]
for row in dataset["text"]:
    # wrapping_print(row)
    wrapping_print(extract_question_prompt(row) + "END")
    print("Answer")
    wrapping_print(extract_answer(row) + "END")
    break
# question = format_question_answer(question, "")

In [ ]:
import logging
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import AutoPeftModelForCausalLM
from transformers.trainer_utils import get_last_checkpoint
from transformers import pipeline
import gradio as gr
# from gradio.pipelines import load_from_pipeline


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    level=logging.INFO,
)

# output_dir = sys.argv[1]
output_dir = "runs/trl_sft"
model_name_or_path = get_last_checkpoint(output_dir)
logger.info(f"Loading model from '{model_name_or_path}'")
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.padding_side = "right"

logger.info(f"Loaded model")


In [ ]:

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, num_return_sequences=2, num_beams=4, do_sample=True, max_new_tokens=50)

In [ ]:
# # ?pipe

row = dataset["text"][0]
wrapping_print(extract_question_prompt(row))

In [ ]:
# for output in pipe(raw_datasets["test"].to_iterable_dataset()):
#     print(output)
#     break

In [ ]:
from functools import partial
from datasets import Dataset

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

to_dataset = lambda iterable_ds: Dataset.from_generator(partial(gen_from_iterable_dataset, iterable_ds), features=iterable_ds.features)

In [ ]:
ds = load_dataset("rotten_tomatoes", split="train", streaming=True)
to_dataset(ds.take(2))[:2], to_dataset(ds.take(2))[:2]

In [ ]:
dataset_dir = "/home/mmordig/reinforcement/datasets/alpha_geo_small"

dataset = load_dataset("csv", data_dir=dataset_dir)#, nrows=10)
next(iter(dataset["train"]))


In [ ]:

# def formatting_func(row):
#     question = row["nl_statement"]
#     answer = row["fl_statement"]
#     return {"text": f"### Question: {question} ### Answer: {answer}"}
# dataset = dataset.map(formatting_func)

def formatting_func_batched(batch):
    questions = batch["nl_statement"]
    answers = batch["fl_statement"]
    return {"text": [f"### Question: {question} ### Answer: {answer}" for (question, answer) in zip(questions, answers)]}

dataset = dataset.map(formatting_func_batched, batched=True)
dataset = dataset.remove_columns(set(dataset["train"].column_names) - {"text"})
dataset = dataset["train"].train_test_split(0.1, seed=1)
dataset_filename = "/home/mmordig/reinforcement/HumbleAttemptAtGeneralAI/geometry_translation/new/runs/verb_dataset"
dataset.save_to_disk(dataset_filename)
dataset, dataset["train"][:2]

In [ ]:
load_dataset(dataset_filename)

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer


dataset_dir = "/home/mmordig/reinforcement/datasets/alpha_geo_small"
model_name_or_path = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left")

tokenizer(" ")["input_ids"][0]


In [ ]:
def get_constant_length_dataset(dataset_dir, **kwargs):
    # use with SFTTrainer together with arg packing=True, not compatible with DataCollatorForCompletionOnlyLM
    
    text_dataset = load_dataset("csv", data_dir=dataset_dir, **kwargs)
    # next(iter(text_dataset["train"]))
    
    def formatting_func(row):
        question = row["nl_statement"]
        answer = row["fl_statement"]
        return f"### Question: {question} ### Answer: {answer}"

    dataset = trl.trainer.ConstantLengthDataset(tokenizer, text_dataset["train"], formatting_func=formatting_func)
    # return dataset
    return DatasetDict({
        "train": dataset,
    })
dataset = get_constant_length_dataset(dataset_dir)
res = next(iter(dataset["train"]))
tokenizer.decode(res["input_ids"]).split("### ")